In [1]:
import os
from glob import glob
from obspy import read, read_events

import numpy as np
import matplotlib.pyplot as plt

from geographiclib.geodesic import Geodesic
from obspy.taup import TauPyModel
import calendar
import radiation

In [7]:
dpi = 300

## Frequencies for filter
fmin = 1/20. #Hz
fmax = 1/10.  #Hzseis

dist_range_0 = 90
dist_range_1 = 100
dist_range_2 = 110
dist_range_3 = 120
dist_range_4 = 130
dist_range_5 = 140

time_min = -40
time_max = 100

linewidth = 0.2
norm_constant = 0.5

if_round = True
switch_yaxis = True
Highlight_SearchArea = True

os.makedirs(f"../Plot/", exist_ok = True)

def PLOTSeismogram(EVENTNAME,atAziBound):
    RePlot = False
    print(EVENTNAME)

    DataStream = read(f"../Data/ProcessedSecondRequest/{EVENTNAME}.PICKLE",format='PICKLE')
    DataStream.filter('bandpass', freqmin=fmin,freqmax=fmax)
    DataStream.differentiate()

    CatFind = glob(f"../Data/CMTSOLUTION/{EVENTNAME[0:12]}*.CMTSOLUTION")
    cat = read_events(CatFind[0])
    EventDepth = cat[0].origins[0].depth/1000.
    EventLat, EventLon = cat[0].origins[0].latitude, cat[0].origins[0].longitude
    EventTime = cat[0].origins[0].time

    catinfo = "%s" %cat
    EventInfo = "%s %.1f km" %(catinfo.split("\n")[-1], EventDepth, )
    print(EventInfo)

    SyntheticLoad = False

    # SearchCenterLat = 64
    # SearchCenterLon = -15
    # EventSearchPair = Geodesic.WGS84.Inverse(EventLat,EventLon,SearchCenterLat,SearchCenterLon, outmask=1929)
    # SearchAzimuth = (EventSearchPair['azi1'] + 360)%360
    
    # Plot data
    fig, axs = plt.subplots(1,5,sharey=True)
    fig.set_size_inches(18,18)   

    # plt.xticks(np.arange(time_min, time_max, step=20))
    plt.yticks(np.arange(0,461,10))

    for ax in axs:
        ax.set_xticks(np.arange(time_min, time_max, step=20))
        ax.set_xlim(time_min,time_max)
        ax.set_xlabel('Time [s]')
        ax.tick_params(axis='both', which='major', labelsize=10, length=5, width=1)

        for basetime in np.arange(time_min, time_max, step=20): ax.axvline(x=basetime,color='grey',linewidth=linewidth, alpha=0.5)

    norm = np.max(DataStream.select(component="T")[0].data) / norm_constant

    azis=[]
    dists=[]
    ratio0_list=[]
    ratio1_list=[]
    SNR_list=[]

    for itrace, trace in enumerate(DataStream.select(component="T")):
        TraceName = trace.stats.network + '.' + trace.stats.station
        dist = trace.stats.distance_in_degrees
        azi = trace.stats.azimuth

        # print(dist, azi)

        if atAziBound == True:
            if azi < 100:
                azi += 360

        # print("dist %.1f azi %.1f" %(dist, azi))
        # Split seismograms by distance range (this needs to be adapted per Event to produce a reasonable plot.
        if dist < dist_range_1 and dist > dist_range_0:
            i_axis = 0
        elif dist < dist_range_2 and dist > dist_range_1:
            i_axis = 1
        elif dist < dist_range_3 and dist > dist_range_2:
            i_axis = 2
        elif dist < dist_range_4 and dist > dist_range_3:
            i_axis = 3
        elif dist < dist_range_5 and dist > dist_range_4:
            i_axis = 4
        else:
            continue

        try:
            phase_time = trace.stats.traveltimes['S'] or trace.stats.traveltimes['Sdiff']
            align_time = phase_time
            # print("align time: ", align_time)
        except:
            print('ERROR loading travetime')
            print(trace)
            continue

        
        if if_round:
            azi = np.round(azi)    
        
        # Plot real data
        plot_time = trace.times(reftime=EventTime)-align_time

        # NormValue = np.max(trace.data[np.logical_and(plot_time>time_min, plot_time<time_max)]) / norm_constant
        try:
            NormValue = np.max(trace.data[np.logical_and(plot_time>time_min, plot_time<time_max)]) / norm_constant
        except:
            continue
        plot_data = trace.data / NormValue + azi
        axs[i_axis].plot(plot_time, plot_data,'k',linewidth=linewidth)

        azis.append(azi) # list all azimuths
        dists.append(dist)

    # # if atAziBound: SearchAzimuth += 360
    # if Highlight_SearchArea:
    #     print("SearchAzimuth", SearchAzimuth)
    #     for ax in axs:
    #         ax.axhline(y=SearchAzimuth,color='yellow',linewidth=linewidth*5,linestyle='--')
    #         ax.fill_between(np.linspace(time_min, time_max,5), SearchAzimuth-10, SearchAzimuth+10, color='yellow', alpha=0.1)
        
    #     if atAziBound:
    #         for ax in axs:
    #             ax.axhline(y=SearchAzimuth+360,color='yellow',linewidth=linewidth*5,linestyle='--')
    #             ax.fill_between(np.linspace(time_min, time_max,5), SearchAzimuth-10+360, SearchAzimuth+10+360, color='yellow', alpha=0.1)
        
    azis = np.array(azis)
    dists = np.array(dists)

    if len(azis) == 0:
        print("no trace at suitable distance!!!")
        return

    # axs[0,0].set_xlim(time_min,time_max)
    axs[0].set_ylim(azis.min()-3,azis.max()+3)

    print("Plot azimuth range: ", azis.max()-azis.min()+6)
    if azis.max()-azis.min() > 300:
        RePlot = True

    axs[0].set_ylabel('Azimuth [$^{\circ}$]')
    axs[0].set_title(' Sdiff dist %d - %d' %(dist_range_0,dist_range_1))
    axs[1].set_title(' Sdiff dist %d - %d' %(dist_range_1,dist_range_2))
    axs[2].set_title(' Sdiff dist %d - %d' %(dist_range_2,dist_range_3))
    axs[3].set_title(' Sdiff dist %d - %d' %(dist_range_3,dist_range_4))
    axs[4].set_title(' Sdiff dist %d - %d' %(dist_range_4,dist_range_5))

    for ax in axs: ax.set_xlabel('Time [s]')

    if switch_yaxis:
        axs[0].invert_yaxis()
    fig.suptitle(EventInfo,fontsize = 20)
    # add radiation pattern
    SearchAzimuth = (azis.max()+azis.min())/2
    radiation_pattern(EVENTNAME, fig, SearchAzimuth)
    
    plt.savefig(f'../Plot/seismogram_{EVENTNAME}.png',format='png')
    plt.close('all')

    return RePlot

In [8]:
def radiation_pattern(EventName, fig, SearchAzimuth):
    # print('%s' %EventName)

    # EVENT = read_events(f"../Data/{EventGroupName}/{EventName}.CMTSOLUTION")[0]
    CatFind = glob(f"../Data/CMTSOLUTION/{EVENTNAME[0:12]}*.CMTSOLUTION")
    EVENT = read_events(CatFind[0])[0]
    #Get a catlogue of events for that month from CMT - this usually works but maybe not always
    try:
        catSearch = read_events('https://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/NEW_MONTHLY/' + str(EventName[0:4]) + '/' +
                                calendar.month_name[int(EventName[4:6])][0:3].lower() + EventName[2:4] + '.ndk')
    except:
        print('HTTP error!!!')
        # continue
        return
    #Loop through the catalogue to find the event - this is done by origin time
    ev = catSearch[np.array([abs(ev.origins[0].time - EVENT.origins[0].time) for ev in catSearch]).argmin()]
    # Preliminry check 
    if abs(ev.origins[0].time - EVENT.origins[0].time) > 100:
        print('Warning!!! CMT source time mismatch: ', abs(ev.origins[0].time - EVENT.origins[0].time), 's')
        #Print event parameters so that user can verify this is the correct event
        print('\n' + ev.event_descriptions[0].text)
        print('Event time:', ev.origins[0].time)
        print('Mw:', [x.mag for x in ev.magnitudes if 'Mw' in x.magnitude_type][0])
        print('Location:', ev.origins[0].latitude, ev.origins[0].longitude)
        print('Depth:',str(ev.origins[0].depth/1000) + 'km\n')
        # continue
        return

    if (ev.origins[0].longitude - EVENT.origins[0].longitude)**2 + (ev.origins[0].latitude - EVENT.origins[0].latitude)**2 > 25:
        print('Warning!!! CMT Location mismatch: ', 'Lat', abs(ev.origins[0].latitude - EVENT.origins[0].latitude), 'Lon', abs(ev.origins[0].longitude - EVENT.origins[0].longitude), 's')
        #Print event parameters so that user can verify this is the correct event
        print('\n' + ev.event_descriptions[0].text)
        print('Event time:', ev.origins[0].time)
        print('Mw:', [x.mag for x in ev.magnitudes if 'Mw' in x.magnitude_type][0])
        print('Location:', ev.origins[0].latitude, ev.origins[0].longitude)
        print('Depth:',str(ev.origins[0].depth/1000) + 'km\n')
        # continue
        return


    EventMag = ev.magnitudes[0].mag
    EventDepth = ev.origins[0].depth/1000
    EventLat, EventLon  = ev.origins[0].latitude, ev.origins[0].longitude

    # # Loading the lat-long data
    # EventPoint = (EventLat, EventLon)
    # StationWet = (station_latitude_wet, station_longitude_wet)

    # # Calculated the distance (in km)
    # dist = great_circle(EventPoint, StationWet).km
    # deg  = kilometers2degrees(dist)
    deg = 105

    #Get the nodal plane parameters from the focal mechanism
    strike = ev.focal_mechanisms[0].nodal_planes.nodal_plane_1.strike
    dip = ev.focal_mechanisms[0].nodal_planes.nodal_plane_1.dip
    rake = ev.focal_mechanisms[0].nodal_planes.nodal_plane_1.rake

    #Calculate the takeoff angle of the ray
    model = TauPyModel('prem')
    arrivals = model.get_travel_times(source_depth_in_km = EventDepth, distance_in_degree = deg,
                                        phase_list = ['Sdiff','S'], receiver_depth_in_km = 0.)   

    takeoff = arrivals[0].takeoff_angle
    #Get radiation patterns
    theta = np.linspace(0,360,361)
    P,S,SH,SV = radiation.rpgen(strike, dip, rake, 0, 0.25, [takeoff], theta)
    MaxSH = np.max(np.abs(SH[0]))

    # Figure Prep.
    # fig = plt.figure(dpi=dpi,figsize=(2.5,2.5))
    # ax = fig.add_subplot(111,projection='polar')
    ax = fig.add_axes([0.888,0.89,0.095,0.095], projection='polar')
    
    ax.plot(np.radians(theta), np.abs(SH[0]))
    ax.set_rmax(np.max(SH[0]))
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_yticklabels([])
    # Fill bewteen the azimuth min and max
    theta_max = np.linspace(SearchAzimuth-10, SearchAzimuth+10,10)
    SH_max = abs(radiation.rpgen(strike, dip, rake, 0, 0.25, [takeoff], theta_max)[2][0])
    ax.fill_between(np.radians(theta_max), SH_max,color='y')

In [4]:
# EVENTNAME = "20050208144821"
# RePlot = PLOTSeismogram(EVENTNAME, atAziBound=False)
# print('PLOTSeismogram %s FINISHED!' %EVENTNAME)

In [9]:
for i, EVENTPATH in enumerate(glob("../Data/ProcessedSecondRequest/*.PICKLE")):
    EVENTNAME = os.path.basename(EVENTPATH).split('.PICKLE')[0]
    try:
        RePlot = PLOTSeismogram(EVENTNAME, atAziBound=False)
        print('PLOTSeismogram %s FINISHED!' %EVENTNAME)
    except:
        print('PLOTSeismogram %s ERROR' %EVENTNAME)
        continue

    if RePlot:
        print("Azimuth at 360 degree boundary, now RePlot!")
        PLOTSeismogram(EVENTNAME, atAziBound=True)

20210822004513
2021-08-22T00:45:13.800000Z | -60.330,  -23.600 | 6.69 mw 12.0 km
Plot azimuth range:  44.0
PLOTSeismogram 20210822004513 FINISHED!
20200322223808
2020-03-22T22:38:08.900000Z |  -4.480, -104.790 | 6.08 mw 12.1 km
Plot azimuth range:  44.0
PLOTSeismogram 20200322223808 FINISHED!
20100524161833
2010-05-24T16:18:33.610000Z |  -8.080,  -71.640 | 6.45 mw 591.4 km
no trace at suitable distance!!!
PLOTSeismogram 20100524161833 FINISHED!
20160425070711
2016-04-25T07:07:11.680000Z | +14.450,  -93.380 | 5.97 mw 17.1 km
Plot azimuth range:  37.0
PLOTSeismogram 20160425070711 FINISHED!
20110403140714
2011-04-03T14:07:14.360000Z | -17.650, -178.450 | 6.4  mw 562.3 km
Plot azimuth range:  11.0
PLOTSeismogram 20110403140714 FINISHED!
20130726070719
2013-07-26T07:07:19.110000Z | -15.300, +167.320 | 6.07 mw 131.0 km
no trace at suitable distance!!!
PLOTSeismogram 20130726070719 FINISHED!
20140426060225
2014-04-26T06:02:25.250000Z | -20.800, -174.290 | 6.14 mw 49.1 km
Plot azimuth range: 

/tmp/ipykernel_1649073/1189734278.py:121: RuntimeWarning: invalid value encountered in divide
  plot_data = trace.data / NormValue + azi


Plot azimuth range:  33.0
PLOTSeismogram 20211218200549 FINISHED!
20220127064011
2022-01-27T06:40:11.560000Z | -19.050, -176.250 | 6.17 mw 14.5 km
Plot azimuth range:  11.0
PLOTSeismogram 20220127064011 FINISHED!
20200318031348
2020-03-18T03:13:48.340000Z | -13.160, +166.870 | 6.09 mw 187.3 km
no trace at suitable distance!!!
PLOTSeismogram 20200318031348 FINISHED!
20220608005548
2022-06-08T00:55:48.460000Z |  -8.980,  -71.250 | 6.48 mw 621.0 km
no trace at suitable distance!!!
PLOTSeismogram 20220608005548 FINISHED!
20220203155859
2022-02-03T15:58:59.920000Z |  -4.500,  -76.950 | 6.44 mw 119.5 km
Plot azimuth range:  14.0
PLOTSeismogram 20220203155859 FINISHED!
20100519105108
2010-05-19T10:51:08.660000Z | -54.710, -135.540 | 6.0  mw 15.3 km
Plot azimuth range:  26.0
PLOTSeismogram 20100519105108 FINISHED!
20100810052353
2010-08-10T05:23:53.880000Z | -17.570, +167.810 | 7.27 mw 31.9 km
no trace at suitable distance!!!
PLOTSeismogram 20100810052353 FINISHED!
20111111104142
2011-11-11T10

/tmp/ipykernel_1649073/1189734278.py:121: RuntimeWarning: invalid value encountered in divide
  plot_data = trace.data / NormValue + azi


Plot azimuth range:  38.0
PLOTSeismogram 20220216071225 FINISHED!
20150222142316
2015-02-22T14:23:16.630000Z | +18.820, -106.850 | 6.24 mw 14.9 km
Plot azimuth range:  37.0
PLOTSeismogram 20150222142316 FINISHED!
20110709150234
2011-07-09T15:02:34.820000Z | -29.150, -176.730 | 5.95 mw 23.1 km
Plot azimuth range:  21.0
PLOTSeismogram 20110709150234 FINISHED!
20220106162510
2022-01-06T16:25:10.900000Z | +11.650,  -87.410 | 6.21 mw 31.3 km


...

Plot azimuth range:  39.0
PLOTSeismogram 20220106162510 FINISHED!
20130905040138
2013-09-05T04:01:38.200000Z | +15.290,  -45.030 | 5.99 mw 12.0 km
Plot azimuth range:  43.0
PLOTSeismogram 20130905040138 FINISHED!
20130813154321
2013-08-13T15:43:21.110000Z |  +5.770,  -78.190 | 6.64 mw 20.9 km
Plot azimuth range:  346.0
PLOTSeismogram 20130813154321 FINISHED!
Azimuth at 360 degree boundary, now RePlot!
20130813154321
2013-08-13T15:43:21.110000Z |  +5.770,  -78.190 | 6.64 mw 20.9 km
Plot azimuth range:  34.0
20100411220815
2010-04-11T22:08:15.530000Z | +37.100,   -3.690 | 6.35 mw 616.5 km
Plot azimuth range:  11.0
PLOTSeismogram 20100411220815 FINISHED!
20210521221323
2021-05-21T22:13:23.820000Z | -16.550, -177.290 | 6.39 mw 12.1 km
Plot azimuth range:  11.0
PLOTSeismogram 20210521221323 FINISHED!
20101225131651
2010-12-25T13:16:51.370000Z | -19.670, +168.040 | 7.3  mw 16.6 km
Plot azimuth range:  6.0
PLOTSeismogram 20101225131651 FINISHED!
20110326224945
2011-03-26T22:49:45.100000Z | -1

/tmp/ipykernel_1649073/1189734278.py:121: RuntimeWarning: invalid value encountered in divide
  plot_data = trace.data / NormValue + azi


Plot azimuth range:  23.0
PLOTSeismogram 20220519101341 FINISHED!
20150422225720
2015-04-22T22:57:20.720000Z | -12.070, +166.330 | 6.25 mw 93.6 km
no trace at suitable distance!!!
PLOTSeismogram 20150422225720 FINISHED!
20210702201441
2021-07-02T20:14:41.260000Z | -21.880, -179.390 | 6.1  mw 607.3 km
Plot azimuth range:  11.0
PLOTSeismogram 20210702201441 FINISHED!
20200805120540
2020-08-05T12:05:40.950000Z | -16.180, +167.910 | 6.49 mw 190.0 km
no trace at suitable distance!!!
PLOTSeismogram 20200805120540 FINISHED!
20180227172929
2018-02-27T17:29:29.970000Z | -60.230, +150.180 | 6.01 mw 12.0 km


/tmp/ipykernel_1649073/1189734278.py:121: RuntimeWarning: invalid value encountered in divide
  plot_data = trace.data / NormValue + azi


Plot azimuth range:  37.0
PLOTSeismogram 20180227172929 FINISHED!
20110301005351
2011-03-01T00:53:51.350000Z | -29.680, -112.260 | 6.05 mw 19.9 km
Plot azimuth range:  32.0
PLOTSeismogram 20110301005351 FINISHED!
20120205001543
2012-02-05T00:15:43.620000Z | -19.020, +168.880 | 6.11 mw 155.6 km
Plot azimuth range:  6.0
PLOTSeismogram 20120205001543 FINISHED!
